In [ ]:
# FFS.ipynb
# Modified by Naomi Trampe
# Last Update: 03-19-2023
# SAMPEL Group

# FFS Tutorial with Langevin Dynamics

#### A tutorial to Forward-Flux Sampling of stochastic systems
Tutorial Breakdown:
* Select the desired potential energy surface for the calculation
* Set FFS hyperparameters and settings
* Equilibrate to settle the system in basin A
* Run the basin simulation to sample first crossings leaving basin A and crossing interface 0
* Run simulations at each interface consecutively to sample the transition path ensemble (TPE)
 * Calculate the rate constant
* Visualize the TPE, paths from individual interfaces, and phase space density of the TPE

#### Learning Objectives

Following the tutorial, the student should know:
1. The general steps in the FFS algorithm
2. The effect of hyperparameters on the performance
    * In general:
       * **interfaces**: Changes the position of the interface order parameter values
       * **basinA**: Changes the position of the interface denoting the extend of the A basin
       * **interface_trajs**: Changes the number of trajectories performed at each interfaces
       * **basinlen**: Changes the length of the basin simulation
       * **lag**: Changes the period of time after a first crossing in which no additional crossings will be collected
    * With interface placement:
       * **n_confs**: Change the number of harvested configurations from the basin simulation
       * **explore_frac**: Changes the fraction of the total interface trajectories to run exploring scouts
       * **p_des**: Changes the desired crossing probability for interface placement
       * **d_min**: Changes the minimum displacement between consecutive interfaces
    * How do these parameters effect the accuracy?
    * How do these parameters effect the efficiency?
3. The effect of the choice of order parameter
    * How does the choice of OP effect the sampling of the TPE?
    * How does the choice of OP effect the accuracy and efficiency?

### Exercises
1. Change

### Potential Energy Surface Options

PES-1: $$ V(x,y) = 0.02(x^4+y^4) - 4\exp(-(x+2)^2 - (y+2)^2) - 4\exp(-(x-2)^2 - (y-2)^2) + 0.3(x-y)^2 + 0.0026 $$

<img src="PES-figures/pes-1.png"> 

PES-2: $$ V(x,y) = 0.03(x^4+y^4) - 4\exp(-(x+2)^2 - (y+2)^2) - 4\exp(-(x-2)^2 - (y-2)^2) + 0.4(x-y)^2 + 4\exp(-(x^2+y^2)) - 2.1245 $$

<img src="PES-figures/pes-2.png">

PES-3: $$ V(x,y) = 0.02(x^4+y^4) - 3.73\exp(-\frac{(x+2)^2}{8} - \frac{(y+2)^2}{8}) - 3.73\exp(-\frac{(x-2)^2}{8} - \frac{(y-2)^2}{8}) + 3\exp(-\frac{x^2}{2} - \frac{y^2}{15}) + 2\exp(-\frac{x^2}{2} - \frac{y^2}{2}) - 0.5085 $$

<img src="PES-figures/pes-3.png">

PES-4: Müller-Brown Potential 
$$ V_{MB}(x,y) = \sum_{i=1}^{4}{A_{i}\exp([ a_{i}(x-\bar{x_{i}})^2 + b_{i}(x-\bar{x_{i}})(y-\bar{y_{i}}) + c_{i}(y-\bar{y_{i}})^2 ])} $$ 
where:

$$ A = [-8, -4, -6.8, 0.6] $$
$$ a = [-0.111, -0.111, -0.722, 0.0778] $$
$$ b = [0, 0, 1.22, 0.0667] $$
$$ c = [-1.11, -1.11, -0.722, 0.0778] $$
$$ \bar{x} = [3, 0, -1.5, -3] $$
$$ \bar{y} = [-3, -1.5, 1.5, 0] $$

<img src="PES-figures/pes-4.png">

### Plot Potential Energy Surface
Here, we will select the PES on which to run FFS.

In [ ]:
import numpy as np
import sys
import math
import langevin_dynamics as ld
import random
import matplotlib.pyplot as plt
from matplotlib import colors, ticker, cm
import time
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
%matplotlib inline

# Select your PES type and parameters: ***** EDIT HERE *****
pes_type = 1                       # type of PES as above
well = 4                           # depth of basin (for PES-1 and PES-2)
basinB_pos = [2.0,2.0]             # coordinates of basinB minimum

# Plot the selected PES
N = 100
x_vec = np.linspace(-3.5, 3.5, N)
y_vec = np.linspace(-3.5, 3.5, N)
energy = np.zeros((N, N))
for i in range(len(x_vec)):
    for j in range(len(y_vec)):
        energy[j][i] = ld.potential(pes_type,x_vec[i],y_vec[j],well=well)
fig, ax = plt.subplots(figsize=(7.5,6))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
ax.set_ylim(-3.5,3.5)
ax.set_xlim(-3.5,3.5)
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.tick_params(axis='both',labelsize=12)
plt.show()

### Define FFS Settings and Parameters
Here, we define a variety of settings and parameters for FFS and Langevin dynamics, such as the OP, various hyperparameters, and Langevin dynamics parameters. 

In [ ]:
# Select your order parameter: ***** EDIT HERE *****
op_xtype = 1                       # order parameter x dependence. 1: linear 2: quadratic 3: cubic ...
op_ytype = 1                       # order parameter y dependence. 1: linear 2: quadratic 3: cubic ...
op_xcoef = [1]                     # order parameter x coefficients. In ascending order [linear, quadratic, cubic, ...]
op_ycoef = [0]                     # order parameter y coefficients. In ascending order [linear, quadratic, cubic, ...]
ld.print_op(op_xtype,op_ytype,op_xcoef,op_ycoef)

# Choose whether to use interface placement algorithms: ***** EDIT HERE ****
place = False                      # True: place interfaces, False: use preselected interfaces

# Place basin B interface or manually select interfaces
space = np.linspace(-3.5,3.5,1000)
sp_lim = max(space)-min(space)
if place:
    basinB = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,basinB_pos[0],basinB_pos[1])
    print("Basin B is at {}\n".format(basinB))
    basinB_inter = [[],[]]
    for i in range(space.shape[0]):
        print(f"Placing... {i+1}/{space.shape[0]}", end="\r")
        for j in range(space.shape[0]):
            if -(sp_lim/space.shape[0]/2) < (abs(ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])))**(1/max(op_xtype,op_ytype))-((abs(basinB))**(1/max(op_xtype,op_ytype))) < (sp_lim/space.shape[0]/2):
                if ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])*basinB > 0:
                    basinB_inter[0].append(space[i])
                    basinB_inter[1].append(space[j])
    print("                                  ")
else:
    # default parameters for op=x and op=y
    ## If using different OP types: ***** EDIT HERE *****
    interfaces = [-1.5, -1.4, -1.3, -1.2, -0.8, -0.5, 0.0, 1.0, 2.0] 
    #interfaces = [-3.25, -3.0, -2.75, -2.5, -2.0, -1.5, 0.0, 2.0, 4.0]
    basinA = -1.78                 # boundary of basin A; must be less than first interface
    #basinA = -3.67
    basinB = interfaces[-1]        # define basin B as the final interface

#FFS settings ***** EDIT HERE *****
if place:
    interfaces = []
    n_confs = 30             # number of collected configurations at interface 0
    explore_frac = 0.2       # fraction of trajectories to launch as exploring scouts
    p_des = 0.3              # desired crossing probability
    d_min = 0.1              # minimum interface displacement
interface_trajs = 1000       # number of trajectories to launch from each interface
basinlen = 50000             # basin simulation time
lag = 500
# Langevin dynamics settings
beta = 2.5                   # 1/kT - increase to run at a lower temperature
gamma = 5                    # friction coefficient
dt = 0.01                    # time step size
# general settings
init_coords = [-1, -1]       # initial coordinates
init_p = [0, 0]              # initial momenta
basineqlen = 5000            # basin equilibration time (in steps)

# Plot PES with defined interfaces
fig, ax = plt.subplots(figsize=(7.5,6))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
ax.set_ylim(-3.5,3.5)
ax.set_xlim(-3.5,3.5)
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.tick_params(axis='both',labelsize=12)
if place:
    ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
else:
    if op_xtype == 1 and op_ycoef == [0]:
        for i in interfaces:
            ax.axvline(x=i,color='k',linewidth=1.5)
        ax.axvline(x=basinB,color='b',linewidth=1.5)
        ax.axvline(x=basinA,color='r',linewidth=1.5)
    elif op_ytype == 1 and op_xcoef == [0]:
        for i in interfaces:
            ax.axhline(y=i,color='k',linewidth=1.5)
        ax.axhline(y=basinB,color='b',linewidth=1.5)
        ax.axhline(y=basinA,color='r',linewidth=1.5)
    elif op_xtype == 1 and op_ytype == 1:
        x = np.linspace(-3.5,3.5,1000)
        for i in interfaces:
            y = i - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='k',linewidth=1.5)
        y = basinB - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='b',linewidth=1.5)
        y = basinA - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='r',linewidth=1.5)
plt.show()

### Equilibrate the System
Here, the initial phasepoint is relaxed into the A basin.

In [ ]:
# declare array to store basin trajectory
basintraj = np.zeros((basinlen + 1, 6),dtype=float)
# calculate initial forces
fx,fy = ld.force(init_coords[0],init_coords[1],init_p[0],init_p[1],dt,beta,gamma,pes_type,well)
# combine positions, momenta, and forces to make an initial phase point
init_phasepoint = init_coords + init_p + [fx,fy]
basintrajeq = ld.vv_step(init_phasepoint,dt,beta,gamma,pes_type,well)
# equilibrate in basin
eqtraj = []
for i in range(1,basineqlen + 1):
    new_basintrajeq = ld.vv_step(basintrajeq,dt,beta,gamma,pes_type,well)
    basintrajeq = new_basintrajeq
    eqtraj.append(basintrajeq)
    op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,basintrajeq[0],basintrajeq[1])
    # check if trajectory reaches basin B
    if op >= basinB:
        sys.exit("Basin trajectory reached B! Exiting...")
# plot equilibration trajectory to ensure adequate equilibration
## Blue line is the equilibration trajectory
## Blue dot is starting point, and red dot is end point
x = [eqtraj[j][0] for j in range(len(eqtraj))]
y = [eqtraj[j][1] for j in range(len(eqtraj))]
fig, ax = plt.subplots(figsize=(7.5,6))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
ax.plot(x,y,color='b',linewidth=1.5)
ax.plot(x[-1],y[-1],'ro')
ax.plot(init_coords[0],init_coords[1],'bo')
if place:
    ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
else:
    if op_xtype == 1 and op_ycoef == [0]:
        for i in interfaces:
            ax.axvline(x=i,color='k',linewidth=1.5)
        ax.axvline(x=basinB,color='b',linewidth=1.5)
        ax.axvline(x=basinA,color='r',linewidth=1.5)
    elif op_ytype == 1 and op_xcoef == [0]:
        for i in interfaces:
            ax.axhline(y=i,color='k',linewidth=1.5)
        ax.axhline(y=basinB,color='b',linewidth=1.5)
        ax.axhline(y=basinA,color='r',linewidth=1.5)
    elif op_xtype == 1 and op_ytype == 1:
        x = np.linspace(-3.5,3.5,1000)
        for i in interfaces:
            y = i - x*op_xcoef[0]/op_ycoef[0]
            ax.plot(x,y,color='k',linewidth=1.5)
        y = basinB - x*op_xcoef[0]/op_ycoef[0]
        ax.plot(x,y,color='b',linewidth=1.5)
        y = basinA - x*op_xcoef[0]/op_ycoef[0]
        ax.plot(x,y,color='r',linewidth=1.5)
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.set_xlim(-3.5,3.5)
ax.set_ylim(-3.5,3.5)
ax.tick_params(axis='both',labelsize=12)
plt.show()


### Run Basin Simulation
Here, we perform the basin simulation to obtain first crossings and the flux through the first interface.
1. Run the simulation for basinlen timesteps

If interfaces are preselected:

2. Count first crossings and harvested crossings

If placing interfaces:

2. Define limit of basin A
3. Find the first interface to achieve desired first crossings

In [ ]:
# start at final point of equilibration
basintraj[0] = basintrajeq
fromBasin = False
first_crosses = []
harvest_crosses = []
n_cross = 0
n_harvest = 0
cross_time = []
harvest_time = []
inter = []
last = -lag
# run basin A simulation
## check for first crossings if not placing interfaces
for j in range(1,basinlen + 1):
    basintraj[j] = ld.vv_step(basintraj[j-1],dt,beta,gamma,pes_type,well)
    op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,basintraj[j][0],basintraj[j][1])
    if not place:
        # collect first crossings|
        if op < basinA:
            fromBasin = True
        if fromBasin == True and op >= interfaces[0]:
            first_crosses.append(basintraj[j])
            n_cross += 1
            if j - last > lag:
                harvest_crosses.append(basintraj[j])
                n_harvest += 1
                last = j
            fromBasin = False
        cross_time.append(n_cross)
        harvest_time.append(n_harvest)
    # check if trajectory reaches basin B
    if op >= basinB:
        sys.exit("Basin trajectory reached B! Exiting...")

# if we are placing interfaces, we need to 
# define basin A and the first interface
# based on the basin trajectory
if place:
    lmda = [ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,basintraj[j][0],basintraj[j][1]) for j in range(len(basintraj))]
    avg = np.mean(lmda)
    stdev = np.std(lmda)
    # define basin A
    basinA = avg + 0.5*stdev
    print("Basin A is at {}\n".format(basinA))
    basinA_inter = [[],[]]
    for i in range(space.shape[0]):
        print(f"Placing... {i+1}/{space.shape[0]}", end="\r")
        for j in range(space.shape[0]):
            if -(sp_lim/space.shape[0]/2) < (abs(ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])))**(1/max(op_xtype,op_ytype))-((abs(basinA))**(1/max(op_xtype,op_ytype))) < (sp_lim/space.shape[0]/2):
                if ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])*basinA > 0:
                    basinA_inter[0].append(space[i])
                    basinA_inter[1].append(space[j])
    interfaces = [0]
    done = False
    # define l0 to satisfy desired harvested crossings
    for i in np.linspace(basinA,max(lmda),int(200*abs(basinA-max(lmda)))):
        n_cross = 0
        n_harvest = 0
        fromBasin = False
        last = -lag
        for j in range(len(lmda)):
            if lmda[j] < basinA:
                fromBasin = True
            if fromBasin == True and lmda[j] >= i:
                n_cross += 1
                fromBasin = False
                if j - last > lag:
                    n_harvest += 1
                    last = j
        if n_harvest >= n_confs:
            interfaces[0] = i
            done = True
    # exit if not enough crossings are harvested
    if not done:
        sys.exit("Not enough harvested crossings. Exiting...")
    print("Interface 0 is at {}\n".format(interfaces[0]))          
    inter_i = [[],[]]
    for i in range(space.shape[0]):
        print(f"Placing... {i+1}/{space.shape[0]}", end="\r")
        for j in range(space.shape[0]):
            if -(sp_lim/space.shape[0]/2) < (abs(ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])))**(1/max(op_xtype,op_ytype))-((abs(interfaces[0]))**(1/max(op_xtype,op_ytype))) < (sp_lim/space.shape[0]/2):
                if ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])*interfaces[0] > 0:
                    inter_i[0].append(space[i])
                    inter_i[1].append(space[j])
    inter.append(inter_i)
    # collect first crossings
    fromBasin = False
    n_cross = 0
    n_harvest = 0
    last = -lag
    for j in range(len(lmda)):
        if lmda[j] < basinA:
            fromBasin = True
        if fromBasin == True and lmda[j] >= interfaces[0]:
            first_crosses.append(basintraj[j])
            n_cross += 1
            if j - last > lag:
                harvest_crosses.append(basintraj[j])
                n_harvest += 1
                last = j
            fromBasin = False
        cross_time.append(n_cross)
        harvest_time.append(n_harvest)

# Plot basin trajectory and new interfaces
x = [basintraj[j][0] for j in range(len(basintraj))]
y = [basintraj[j][1] for j in range(len(basintraj))]
# plot the configs selected for the first interface
fig, ax = plt.subplots(figsize=(7.5,6))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
ax.plot(x,y,color='b')
if place:
    for i in range(len(inter)):
        ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
    ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
    ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
else:
    if op_xtype == 1 and op_ycoef == [0]:
        for i in interfaces:
            ax.axvline(x=i,color='k',linewidth=1.5)
        ax.axvline(x=basinB,color='b',linewidth=1.5)
        ax.axvline(x=basinA,color='r',linewidth=1.5)
    elif op_ytype == 1 and op_xcoef == [0]:
        for i in interfaces:
            ax.axhline(y=i,color='k',linewidth=1.5)
        ax.axhline(y=basinB,color='b',linewidth=1.5)
        ax.axhline(y=basinA,color='r',linewidth=1.5)
    elif op_xtype == 1 and op_ytype == 1:
        x = np.linspace(-3.5,3.5,1000)
        for i in interfaces:
            y = i - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='k',linewidth=1.5)
        y = basinB - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='b',linewidth=1.5)
        y = basinA - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='r',linewidth=1.5)
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.set_xlim(-3.5,3.5)
ax.set_ylim(-3.5,3.5)
ax.tick_params(axis='both',labelsize=12)
plt.show()

### Select Configurations for Interface Trajectories
Here, we evenly distribute the harvested configurations between interface simulations

In [ ]:
# exit if there are no first crossings
if n_cross == 0:
    sys.exit("No first crossings obtained from basin A to interface 0. Exiting...")
flux = n_cross/(basinlen*dt)
print("Flux through first interface: {}\n".format(flux))
# evenly distribute configurations through simulations
print("Number of first crossings: {}\n".format(len(first_crosses)))
configs = []
configs_list = []
configs_group = []
configs_groups = []
for i in range(len(first_crosses)):
    configs_group.append(0)
configs_groups.append(configs_group)
alltraj_group = []
alltraj_groups = []
for j in range(len(first_crosses)):
    for i in range(int(interface_trajs/len(first_crosses))):
        configs.append(first_crosses[j])
        alltraj_group.append(j)
# randomly select the remainder without replacement
for i in np.asarray(random.sample(range(len(first_crosses)),k=interface_trajs%len(first_crosses))):
    configs.append(first_crosses[i])
    alltraj_group.append(i)
configs = np.asarray(configs)
configs_list.append(configs)
alltraj_groups.append(alltraj_group)


# plot the configs selected for the first interface
fig, ax = plt.subplots(figsize=(7.5,6))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
ax.plot(x,y,color='b')
if place:
    for i in range(len(inter)):
        ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
    ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
    ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
else:
    if op_xtype == 1 and op_ycoef == [0]:
        for i in interfaces:
            ax.axvline(x=i,color='k',linewidth=1.5)
        ax.axvline(x=basinB,color='b',linewidth=1.5)
        ax.axvline(x=basinA,color='r',linewidth=1.5)
    elif op_ytype == 1 and op_xcoef == [0]:
        for i in interfaces:
            ax.axhline(y=i,color='k',linewidth=1.5)
        ax.axhline(y=basinB,color='b',linewidth=1.5)
        ax.axhline(y=basinA,color='r',linewidth=1.5)
    elif op_xtype == 1 and op_ytype == 1:
        x = np.linspace(-3.5,3.5,1000)
        for i in interfaces:
            y = i - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='k',linewidth=1.5)
        y = basinB - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='b',linewidth=1.5)
        y = basinA - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='r',linewidth=1.5)
ax.plot(configs[:,0],configs[:,1],'ro')
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.tick_params(axis='both',labelsize=12)
ax.set_xlim(-3.5,3.5)
ax.set_ylim(-3.5,3.5)
plt.show()
#Plot harvested and total first crossings across the timesteps
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(cross_time,label="Total")
ax.plot(harvest_time,label="Harvested")
ax.set_xlabel('Timestep',fontsize=15)
ax.set_ylabel('# of Configurations',fontsize=15)
ax.tick_params(axis='both',labelsize=12)
ax.legend(fontsize=13)
plt.show()

### Interface Simulations
Here, we run simulations at each interface sequentially until basin B is reached, capturing the crossing probabilities and the lists of all simulated paths.

If using preselected interfaces:

1. Simulate trajectories from each selected configuration
2. Collect configurations crossing the next interface

Repeat until basin B is reached

If placing interfaces:

1. Run exploring scouts on a fraction of the total interface trajectories
2. Determine the location of the next interface
3. Simulate trajectories from each selected configuration
4. Collect configurations crossing the next interface

Repeat until basin B is reached

In [ ]:
# run interfaces sequentially
starttime = time.monotonic()
alltrajs = []
cross_probs = []
allyestrajs = []
allnotrajs = []
if not place:
    for i in range(len(interfaces) - 1):
        fig, ax = plt.subplots(figsize=(7.5,6))
        inttrajs = []
        yestrajs = []
        notrajs = []
        configs_group = []
        print("Starting interface {}...".format(i))
        first_crosses = []
        n_cross = 0
        # run simulations for each selected configuration
        for config in range(len(configs)):
            op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,configs[config][0],configs[config][1])
            step = 0
            traj = []
            traj.append(configs[config])
            while op >= basinA and op < interfaces[i+1]:
                traj.append(ld.vv_step(traj[step],dt,beta,gamma,pes_type,well))
                step += 1
                op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,traj[step][0],traj[step][1])
            if op >= interfaces[i+1]:
                n_cross += 1
                first_crosses.append(traj[step])
                configs_group.append(config)
                yestrajs.append(np.asarray(traj))
            else:
                notrajs.append(np.asarray(traj))
            inttrajs.append(np.asarray(traj))
        if n_cross == 0:
            sys.exit("No first crossings obtained from interface {} to interface {}. Exiting...".format(i,i+1))
        # determine the crossing probability
        cross_prob = n_cross/(len(inttrajs))
        cross_probs.append(cross_prob)
        alltrajs.append(inttrajs)
        allyestrajs.append(yestrajs)
        allnotrajs.append(notrajs)
        print("Interface {} first crossings from {}: {}".format(i+1,i,len(first_crosses)))
        print("{} to {} crossing prob: {}\n".format(i,i+1,cross_prob))
        configs = []
        alltraj_group = []
        for j in range(len(first_crosses)):
            for i in range(int(interface_trajs/len(first_crosses))):
                configs.append(first_crosses[j])
                alltraj_group.append(j)
        # randomly select the remainder without replacement
        for i in np.asarray(random.sample(range(len(first_crosses)),k=interface_trajs%len(first_crosses))):
            configs.append(first_crosses[i])
            alltraj_group.append(i)
        configs = np.asarray(configs)
        configs_list.append(configs)
        configs_groups.append(configs_group)
        alltraj_groups.append(alltraj_group)
        # plot contour for collected configurations
        ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
        cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
        cbar.set_ticks([])
        cbar.set_label(label = 'Energy', size=12)
        if op_xtype == 1 and op_ycoef == [0]:
            for i in interfaces:
                ax.axvline(x=i,color='k',linewidth=1.5)
            ax.axvline(x=basinB,color='b',linewidth=1.5)
            ax.axvline(x=basinA,color='r',linewidth=1.5)
        elif op_ytype == 1 and op_xcoef == [0]:
            for i in interfaces:
                ax.axhline(y=i,color='k',linewidth=1.5)
            ax.axhline(y=basinB,color='b',linewidth=1.5)
            ax.axhline(y=basinA,color='r',linewidth=1.5)
        elif op_xtype == 1 and op_ytype == 1:
            x = np.linspace(-3.5,3.5,1000)
            for i in interfaces:
                y = i - x*op_xcoef/op_ycoef
                ax.plot(x,y,color='k',linewidth=1.5)
            y = basinB - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='b',linewidth=1.5)
            y = basinA - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='r',linewidth=1.5)
        ax.set_xlabel('x',fontsize=15)
        ax.set_ylabel('y',fontsize=15)
        ax.set_xlim(-3.5,3.5)
        ax.set_ylim(-3.5,3.5)
        ax.tick_params(axis='both',labelsize=12)
        ax.plot(configs[:,0],configs[:,1],'ro')
        plt.show()
else:
    cnt = 0
    while interfaces[-1] < basinB:
        ##EXPLORING SCOUTS##
        print("Exploring from interface {}...".format(cnt))
        op_max = []
        for config in configs[::int(1/explore_frac)]:
            op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,config[0],config[1])
            op_maxi = op
            step = 0
            traj = []
            traj.append(config)
            while op >= basinA and op < basinB and step < 5000:
                traj.append(ld.vv_step(traj[step],dt,beta,gamma,pes_type,well))
                step += 1
                op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,traj[step][0],traj[step][1])
                if op > op_maxi:
                    op_maxi = op
            op_max.append(op_maxi)
        op_max.sort()
        if op_max[int((1-p_des)*len(op_max))] > basinB:
            interfaces.append(basinB)
        elif op_max[int((1-p_des)*len(op_max))]-interfaces[-1] < d_min:
            interfaces.append(interfaces[-1]+d_min)
        else:
            interfaces.append(op_max[int((1-p_des)*len(op_max))])
        print("Next interface is at {}\n".format(interfaces[-1]))
        inter_i = [[],[]]
        for i in range(space.shape[0]):
            print(f"Placing... {i+1}/{space.shape[0]}", end="\r")
            for j in range(space.shape[0]):
                if -(sp_lim/space.shape[0]/2) < (abs(ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])))**(1/max(op_xtype,op_ytype))-((abs(interfaces[cnt+1]))**(1/max(op_xtype,op_ytype))) < (sp_lim/space.shape[0]/2):
                    if ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,space[i],space[j])*interfaces[cnt+1] > 0:
                        inter_i[0].append(space[i])
                        inter_i[1].append(space[j])
        inter.append(inter_i)
        inttrajs = []
        yestrajs = []
        notrajs = []
        configs_group = []
        ##INTERFACE SIMULATION##
        print("Starting interface {}...".format(cnt))
        first_crosses = []
        n_cross = 0
        for config in range(len(configs)):
            op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,configs[config][0],configs[config][1])
            step = 0
            traj = []
            traj.append(configs[config])
            while op >= basinA and op < interfaces[cnt+1]:
                traj.append(ld.vv_step(traj[step],dt,beta,gamma,pes_type,well))
                step += 1
                op = ld.calc_op_f(op_xtype,op_ytype,op_xcoef,op_ycoef,traj[step][0],traj[step][1])
            if op >= interfaces[cnt+1]:
                n_cross += 1
                first_crosses.append(traj[step])
                configs_group.append(config)
                yestrajs.append(np.asarray(traj))
            else:
                notrajs.append(np.asarray(traj))
            inttrajs.append(np.asarray(traj))
        if n_cross == 0:
            sys.exit("No first crossings obtained from interface {} to interface {}. Exiting...".format(cnt,cnt+1))
        cross_prob = n_cross/(len(inttrajs))
        cross_probs.append(cross_prob)
        alltrajs.append(inttrajs)
        allyestrajs.append(yestrajs)
        allnotrajs.append(notrajs)
        print("Interface {} first crossings from {}: {}".format(cnt+1,cnt,len(first_crosses)))
        print("{} to {} crossing prob: {}\n\n".format(cnt,cnt+1,cross_prob))
        configs = []
        alltraj_group = []
        for j in range(len(first_crosses)):
            for i in range(int(interface_trajs/len(first_crosses))):
                configs.append(first_crosses[j])
                alltraj_group.append(j)
        # randomly select the remainder without replacement
        for i in np.asarray(random.sample(range(len(first_crosses)),k=interface_trajs%len(first_crosses))):
            configs.append(first_crosses[i])
            alltraj_group.append(i)
        configs = np.asarray(configs)
        configs_list.append(configs)
        configs_groups.append(configs_group)
        alltraj_groups.append(alltraj_group)
        fig, ax = plt.subplots(figsize=(7.5,6))
        cnt += 1
        ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
        cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
        cbar.set_ticks([])
        cbar.set_label(label = 'Energy', size=12)
        for i in range(len(inter)):
            ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
        ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
        ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
        ax.plot(configs[:,0],configs[:,1],'ro')
        ax.set_xlabel('x',fontsize=15)
        ax.set_ylabel('y',fontsize=15)
        ax.tick_params(axis='both',labelsize=12)
        ax.plot(configs[:,0],configs[:,1],'ro',markersize=1.5)
        plt.show()
        if cnt > 30:
            sys.exit("Too many interfaces placed. OP may not be able to sample from A to B. Exiting...")
print("Done!")
endtime = time.monotonic()

### Calculate the Rate Constant
Here, we calculate the rate constant based on the effective positive flux:

$$k_{AB}=\Phi_0\prod_{i=0}^{n}P\left(\lambda_{i+1}|\lambda_{i}\right)$$

In [ ]:
print("Total Simulation Time: %.2f s" % (endtime-starttime))
        
rate = flux*np.prod(np.asarray(cross_probs))
print('Rate = %8.3e' % rate)



# plot cumulative crossing probability
cumuprob = []
for i in range(0,len(cross_probs)+1):
    cumuprob.append(np.prod(np.asarray(cross_probs[:i])))
probs = plt.figure(2,figsize=(6,4))
plt.plot(interfaces,cumuprob,color='b')
plt.yscale('log')
plt.xlabel('$\lambda_i$',fontsize=15)
plt.ylabel('$P(\lambda_i|\lambda_0)$',fontsize=15)
ax.tick_params(axis='both',labelsize=12)
plt.show()

### Build TPE
Here, we stitch together partial paths between each interface to obtain continuous paths between the basins which make up the transition path ensemble (TPE)

In [ ]:
TPE = []
for i in range(len(configs_groups[-1])): 
    complete_path = []
    prev_traj = configs_groups[-1][i]
    complete_path.append(alltrajs[-1][prev_traj])
    for j in range(2,len(interfaces)):
        conf = alltraj_groups[-j][prev_traj]
        prev_traj = configs_groups[-j][conf]
        complete_path.append(alltrajs[-j][prev_traj])
    TPE.append(complete_path)

colors = ['red','orange','yellow','green','blue','indigo','violet']
    
fig, ax = plt.subplots(figsize=(11.25,9))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
for i in range(len(TPE[::1])):
    for j in range(len(TPE[i])):
        ax.plot(TPE[1*i][j][:,0],TPE[1*i][j][:,1],color=colors[j%len(colors)])
ax.set_xlabel('x',fontsize=20)
ax.set_ylabel('y',fontsize=20)
ax.set_xlim(-3.5,3.5)
ax.set_ylim(-3.5,3.5)
ax.tick_params(axis='both',labelsize=16)
if place:
    for i in range(len(inter)):
        ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
    ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
    ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
else:
    if op_xtype == 1 and op_ycoef == [0]:
        for i in interfaces:
            ax.axvline(x=i,color='k',linewidth=1.5)
        ax.axvline(x=basinB,color='b',linewidth=1.5)
        ax.axvline(x=basinA,color='r',linewidth=1.5)
    elif op_ytype == 1 and op_xcoef == [0]:
        for i in interfaces:
            ax.axhline(y=i,color='k',linewidth=1.5)
        ax.axhline(y=basinB,color='b',linewidth=1.5)
        ax.axhline(y=basinA,color='r',linewidth=1.5)
    elif op_xtype == 1 and op_ytype == 1:
        x = np.linspace(-3.5,3.5,1000)
        for i in interfaces:
            y = i - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='k',linewidth=1.5)
        y = basinB - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='b',linewidth=1.5)
        y = basinA - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='r',linewidth=1.5)
plt.show()


### Plot All Sampled Paths

In [ ]:
skip = 1
for i in range(len(interfaces) - 1):
    fig, ax = plt.subplots(figsize=(7.5,6))
    ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
    cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
    cbar.set_ticks([])
    cbar.set_label(label = 'Energy', size=12)
    for j in range(len(alltrajs[i][::skip])):
        ax.plot(alltrajs[i][skip*j][:,0],alltrajs[i][skip*j][:,1],linewidth=1.5,color=colors[-(i-1)%len(colors)])
    if place:
        for i in range(len(inter)):
            ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
        ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
        ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
    else:
        if op_xtype == 1 and op_ycoef == [0]:
            for i in interfaces:
                ax.axvline(x=i,color='k',linewidth=1.5)
            ax.axvline(x=basinB,color='b',linewidth=1.5)
            ax.axvline(x=basinA,color='r',linewidth=1.5)
        elif op_ytype == 1 and op_xcoef == [0]:
            for i in interfaces:
                ax.axhline(y=i,color='k',linewidth=1.5)
            ax.axhline(y=basinB,color='b',linewidth=1.5)
            ax.axhline(y=basinA,color='r',linewidth=1.5)
        elif op_xtype == 1 and op_ytype == 1:
            x = np.linspace(-3.5,3.5,1000)
            for i in interfaces:
                y = i - x*op_xcoef/op_ycoef
                ax.plot(x,y,color='k',linewidth=1.5)
            y = basinB - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='b',linewidth=1.5)
            y = basinA - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='r',linewidth=1.5)
    ax.set_xlabel('x',fontsize=15)
    ax.set_ylabel('y',fontsize=15)
    ax.set_xlim(-3.5,3.5)
    ax.set_ylim(-3.5,3.5)
    ax.tick_params(axis='both',labelsize=12)
    plt.show()

### Plot All Successful Paths

In [ ]:
skip = 1
for i in range(len(interfaces) - 1):
    fig, ax = plt.subplots(figsize=(7.5,6))
    ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
    cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
    cbar.set_ticks([])
    cbar.set_label(label = 'Energy', size=12)
    for j in range(len(allyestrajs[i][::skip])):
        ax.plot(allyestrajs[i][skip*j][:,0],allyestrajs[i][skip*j][:,1],linewidth=1.5,color=colors[-(i-1)%len(colors)])
    if place:
        for i in range(len(inter)):
            ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
        ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
        ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
    else:
        if op_xtype == 1 and op_ycoef == [0]:
            for i in interfaces:
                ax.axvline(x=i,color='k',linewidth=1.5)
            ax.axvline(x=basinB,color='b',linewidth=1.5)
            ax.axvline(x=basinA,color='r',linewidth=1.5)
        elif op_ytype == 1 and op_xcoef == [0]:
            for i in interfaces:
                ax.axhline(y=i,color='k',linewidth=1.5)
            ax.axhline(y=basinB,color='b',linewidth=1.5)
            ax.axhline(y=basinA,color='r',linewidth=1.5)
        elif op_xtype == 1 and op_ytype == 1:
            x = np.linspace(-3.5,3.5,1000)
            for i in interfaces:
                y = i - x*op_xcoef/op_ycoef
                ax.plot(x,y,color='k',linewidth=1.5)
            y = basinB - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='b',linewidth=1.5)
            y = basinA - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='r',linewidth=1.5)
    ax.set_xlabel('x',fontsize=15)
    ax.set_ylabel('y',fontsize=15)
    ax.set_xlim(-3.5,3.5)
    ax.set_ylim(-3.5,3.5)
    ax.tick_params(axis='both',labelsize=12)
    plt.show()

### Plot All Unsuccessful Paths

In [ ]:
skip = 1
for i in range(len(interfaces) - 1):
    fig, ax = plt.subplots(figsize=(7.5,6))
    ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,21), cmap = 'jet',linewidths=1.5)
    cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'),ax=ax)
    cbar.set_ticks([])
    cbar.set_label(label = 'Energy', size=12)
    for j in range(len(allnotrajs[i][::skip])):
        ax.plot(allnotrajs[i][skip*j][:,0],allnotrajs[i][skip*j][:,1],linewidth=1.5,color=colors[-(i-1)%len(colors)])
    if place:
        for i in range(len(inter)):
            ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.5)
        ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.5)
        ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.5)
    else:
        if op_xtype == 1 and op_ycoef == [0]:
            for i in interfaces:
                ax.axvline(x=i,color='k',linewidth=1.5)
            ax.axvline(x=basinB,color='b',linewidth=1.5)
            ax.axvline(x=basinA,color='r',linewidth=1.5)
        elif op_ytype == 1 and op_xcoef == [0]:
            for i in interfaces:
                ax.axhline(y=i,color='k',linewidth=1.5)
            ax.axhline(y=basinB,color='b',linewidth=1.5)
            ax.axhline(y=basinA,color='r',linewidth=1.5)
        elif op_xtype == 1 and op_ytype == 1:
            x = np.linspace(-3.5,3.5,1000)
            for i in interfaces:
                y = i - x*op_xcoef/op_ycoef
                ax.plot(x,y,color='k',linewidth=1.5)
            y = basinB - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='b',linewidth=1.5)
            y = basinA - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='r',linewidth=1.5)
    ax.set_xlabel('x',fontsize=15)
    ax.set_ylabel('y',fontsize=15)
    ax.set_xlim(-3.5,3.5)
    ax.set_ylim(-3.5,3.5)
    ax.tick_params(axis='both',labelsize=12)
    plt.show()

### Plot TPE Density
Here, we plot the density of configurations from the TPE

In [ ]:
skip = 1
x = []
y = []
for i in range(len(TPE)):
    for j in range(len(TPE[i])):
        for k in TPE[i][j][:,0]:
            x.append(k)
        for k in TPE[i][j][:,1]:
            y.append(k)
y = np.asarray(y)
x = np.asarray(x)


bins = 20
sort = True
fig , ax = plt.subplots(figsize=(7.5,6))
ax.contour(x_vec,y_vec,energy,np.linspace(-3,3,15), colors='silver')
data , x_e, y_e = np.histogram2d( x, y, bins = bins, density = True )
z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

#To be sure to plot all data
z[np.where(np.isnan(z))] = 0.0

# Sort the points by density, so that the densest points are plotted last
if sort :
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

m = 'jet'
ax.scatter( x, y, c=z, s=2, cmap = m, alpha = 0.002)

norm = Normalize(vmin = np.min(z), vmax = np.max(z))

cbar = fig.colorbar(cm.ScalarMappable(norm = norm, cmap = m), ax=ax)
cbar.set_ticks([])
cbar.ax.set_ylabel('Density',size=12)         
ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.set_xlim(-3.5,3.5)
ax.set_ylim(-3.5,3.5)
ax.tick_params(axis='both',labelsize=12)
if place:
    for i in range(len(inter)):
        ax.plot(inter[i][0],inter[i][1],'ko',markersize=0.25)
    ax.plot(basinA_inter[0],basinA_inter[1],'ro',markersize=0.25)
    ax.plot(basinB_inter[0],basinB_inter[1],'bo',markersize=0.25)
else:
    if op_xtype == 1 and op_ycoef == [0]:
        for i in interfaces:
            ax.axvline(x=i,color='k',linewidth=1.5,alpha=0.5)
        ax.axvline(x=basinB,color='b',linewidth=1.5,alpha=0.5)
        ax.axvline(x=basinA,color='r',linewidth=1.5,alpha=0.5)
    elif op_ytype == 1 and op_xcoef == [0]:
        for i in interfaces:
            ax.axhline(y=i,color='k',linewidth=1.5,alpha=0.5)
        ax.axhline(y=basinB,color='b',linewidth=1.5,alpha=0.5)
        ax.axhline(y=basinA,color='r',linewidth=1.5,alpha=0.5)
    elif op_xtype == 1 and op_ytype == 1:
        x = np.linspace(-3.5,3.5,1000)
        for i in interfaces:
            y = i - x*op_xcoef/op_ycoef
            ax.plot(x,y,color='k',linewidth=1.5,alpha=0.5)
        y = basinB - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='b',linewidth=1.5,alpha=0.5)
        y = basinA - x*op_xcoef/op_ycoef
        ax.plot(x,y,color='r',linewidth=1.5,alpha=0.5)
plt.show()